In [1]:
import pandas as pd
import numpy as np

In [2]:
LD_50 = pd.read_csv('../data/raw/ChEMBL_LD50.csv', delimiter=';')
LD_50.shape

(13758, 47)

In [3]:
# Проверим, есть ли в датасете ПОЛНЫЕ дубли
duplicates = LD_50.duplicated()
duplicated_rows = LD_50[duplicates]
print(duplicates.sum()) # видно, что в датасете есть 5 строк, которые являются дублями
# Удалим их
LD_50.drop_duplicates(inplace=True)

5


In [4]:
# Выберем из датасета только те столбцы, которые понадобятся для дальнейшего анализа
LD_50 = LD_50[['Smiles', 
         'Standard Type',          
         'Standard Value', 
         'Molecular Weight',
         'Standard Units', 
         'Target Name', 
         'Assay Description',
         'Target Organism', 
         'Target Type'
         ]]

In [5]:
print(f'Количество уникальных Smiles: {LD_50['Smiles'].nunique()}') # Из 13626 соединений, для которых приведены Smiles, только 9531 уникальное
# Посмотрим, чем отличаются строки с дублирующимися Smiles
d_smiles = LD_50.duplicated(subset='Smiles', keep=False)
duplicated_rows = LD_50[d_smiles]
duplicated_rows[duplicated_rows['Smiles'] == 'CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21']

Количество уникальных Smiles: 9531


,Smiles,Standard Type,Standard Value,Molecular Weight,Standard Units,Target Name,Assay Description,Target Organism,Target Type
1096,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Rattus norvegicus,Lethal dose of compound after peroral administ...,Rattus norvegicus,ORGANISM
3031,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Rattus norvegicus,Lethal dose was measured in rat after oral adm...,Rattus norvegicus,ORGANISM
3871,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,314.0,319.34,mg.kg-1,Rattus norvegicus,Lethal dose was measured in rat after iv admin...,Rattus norvegicus,ORGANISM
4076,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Mus musculus,In vivo acute toxicity against Escherichia col...,Mus musculus,ORGANISM
5395,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,2500.0,319.34,mg.kg-1,Mus musculus,Lethal dose of compound in OF1-strain female s...,Mus musculus,ORGANISM
7535,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Mus musculus,Acute toxicity on oral administration in mice,Mus musculus,ORGANISM
8476,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,314.0,319.34,mg.kg-1,Mus musculus,Acute toxicity on intravenous injection in in ...,Mus musculus,ORGANISM
9586,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,327.0,319.34,mg.kg-1,Mus musculus,In vivo acute toxicity against Escherichia col...,Mus musculus,ORGANISM
11750,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,ADMET,Toxicity in po dosed mouse,NaN,ADMET
13139,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,ADMET,Toxicity in po dosed CF mouse measured after 1...,NaN,ADMET


Попробуем определить способ введения вещества в организм из названия публикации об исследовании

In [6]:
import re 

def adm_type_parsing(x): 
        if pd.notna(x):
            ip = r'\bip\b|intraperit'
            if re.search(r'\bip\b|intraperit', x, re.IGNORECASE):
                return 'Intraperitoneal'
            elif re.search(r'\biv\b|intrav', x, re.IGNORECASE):
                return 'Intravenous'
            elif re.search(r'oral|perorla', x, re.IGNORECASE):
                return 'Oral'
            elif re.search(r'cell',  x, re.IGNORECASE):
                return 'In vitro'
            elif re.search(r'muscular', x, re.IGNORECASE):
                return 'Intramusculary'
        else:
            return np.nan   
        
LD_50['Administration type'] = LD_50['Assay Description'].apply(adm_type_parsing)
LD_50.drop('Assay Description', axis=1, inplace=True)

In [18]:
# Посмотрим, чем отличаются строки с дублирующимися Smiles
d_smiles = LD_50.duplicated(subset='Smiles', keep=False)
duplicated_rows = LD_50[d_smiles]
print(f'Количество строк, в которых дублируется Smiles соединения: {d_smiles.sum()}')
duplicated_rows[duplicated_rows['Smiles'] == 'CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21']

Количество строк, в которых дублируется Smiles соединения: 5061


,Smiles,Standard Type,Standard Value,Molecular Weight,Standard Units,Target Name,Administration type,LD_50
1096,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Rattus norvegicus,Oral,4000.0
3031,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Rattus norvegicus,Oral,4000.0
3871,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,314.0,319.34,mg.kg-1,Rattus norvegicus,Intravenous,314.0
4076,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Mus musculus,Oral,4000.0
5395,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,2500.0,319.34,mg.kg-1,Mus musculus,Oral,2500.0
7535,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,Mus musculus,Oral,4000.0
8476,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,314.0,319.34,mg.kg-1,Mus musculus,Intravenous,314.0
9586,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,327.0,319.34,mg.kg-1,Mus musculus,Intravenous,327.0
11750,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,ADMET,None,4000.0
13139,CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21,LD50,4000.0,319.34,mg.kg-1,ADMET,None,4000.0


Удалим строки, в которых есть пропуски в ключевых столбцах: Smiles и Standard Value

In [20]:
# Удалим строки, для которых отсутствуют значения LD50
LD_50 = LD_50[pd.notna(LD_50['Standard Value'])]
# Удалим строки, для которых отсутствуют значения Smiles
LD_50 = LD_50[pd.notna(LD_50['Smiles'])]
# Посмотрим на результат
LD_50.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11957 entries, 0 to 13757
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Smiles               11957 non-null  object 
 1   Standard Type        11957 non-null  object 
 2   Standard Value       11957 non-null  float64
 3   Molecular Weight     11957 non-null  float64
 4   Standard Units       11957 non-null  object 
 5   Target Name          11957 non-null  object 
 6   Administration type  8140 non-null   object 
 7   LD_50                11957 non-null  float64
dtypes: float64(3), object(5)
memory usage: 840.7+ KB


In [9]:
# Заменим данные в столбце Target Name на данные из столбца Target Organism для тех случаев, где соединение тестировалось на людях 
LD_50[LD_50['Target Organism'] == 'Homo sapiens']['Target Name'] = LD_50['Target Organism']
# Теперь можно удалить столбцы Target Organism и Target Type
LD_50.drop(['Target Organism', 'Target Type'], axis=1, inplace=True)

/tmp/ipykernel_50132/3601995115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LD_50[LD_50['Target Organism'] == 'Homo sapiens']['Target Name'] = LD_50['Target Organism']


Теперь приведем все единицы измерения к одному виду

In [10]:
LD_50['Standard Units'].value_counts()

Standard Units
mg.kg-1                    6579
uM                         3413
umol.kg-1                   783
ug ml-1                     677
ug                          395
mM                          164
nM                          118
ppm                          94
g/Kg                         90
microgAi/g                   61
mg/L                         58
M                            48
g/ha                         40
microg/cm2                   35
umol/L                       27
mg kg-1 i.p.                 26
ng                           24
M kg-1                       24
uM/L                         22
mg/kg/day                    21
p.p.m.                       20
10'-3M                       19
10'-5M                       18
mg/ml                        14
ug/mg                        14
10'-6M                       14
mg                           13
mg/m2                        13
mg kg-1 day-1                13
uM/ml                        12
10'-4M                   

In [11]:
def units_calc(row, df):
    if pd.notna(row['Standard Units']):
        if row['Standard Units'] in ('uM', 'umol/L', 'uM/L'):
            mg_kg = row['Standard Value'] * 10e-3 * row['Molecular Weight'] * 0.7 # если вещество равномерно распределяется по всей воде организма, можно взять Vd ≈ 0.7 L/kg            
        elif row['Standard Units'] == 'umol.kg-1':
            mg_kg = row['Standard Value'] * 10e-3 * row['Molecular Weight']           
        elif row['Standard Units'] == 'ug ml-1':
            mg_kg = row['Standard Value'] * 10e-3 * 0.7            
        elif row['Standard Units'] == 'nM':
            mg_kg = row['Standard Value'] * 10e-6 * row['Molecular Weight'] * 0.7            
        elif row['Standard Units'] in ('mM', 'uM/ml'):
            mg_kg = row['Standard Value'] * row['Molecular Weight'] * 0.7            
        elif row['Standard Units'] == 'M':
            mg_kg = row['Standard Value'] * row['Molecular Weight'] * 10e3 * 0.7            
        elif row['Standard Units'] == 'g/Kg':
            mg_kg = row['Standard Value'] * 10e3       
        elif row['Standard Units'] == 'M kg-1':
            mg_kg = row['Standard Value'] * row['Molecular Weight'] * 10e3          
        elif row['Standard Units'] == 'mg.kg-1':
            mg_kg = row['Standard Value']
        else:
            return np.nan
        return mg_kg
    else:       
        return np.nan

LD_50['LD_50'] = LD_50.apply(units_calc, df=LD_50, axis=1)

In [12]:
LD_50.sample(10)

,Smiles,Standard Type,Standard Value,Molecular Weight,Standard Units,Target Name,Administration type,LD_50
7119,O=c1[nH]c(=O)n(C2CCCO2)cc1F,LD50,860.0,200.17,mg.kg-1,Mus musculus,None,860.000000
4861,C=C(c1cc(C)cc(C)c1OCC(O)CNC(C)(C)C)n1ccnc1,LD50,15.9,343.47,mg.kg-1,Mus musculus,None,15.900000
4826,C=CC(C)(C)c1c(OC)cc(O)c2c(=O)c3c(CC=C(C)C)c(O)...,LD50,16.3,410.47,uM,A549,In vitro,46.834627
6347,COCCN(CCOC)C(=O)c1cc2cc3nc(cc4[nH]c(cc5nc(cc1[...,LD50,60.0,688.87,nM,HeLa,In vitro,0.289325
4639,Cc1c2c(=O)[nH]n(CCN3CCN(C/C=C/c4ccccc4)CC3)c(=...,LD50,2000.0,483.62,mg.kg-1,Mus musculus,Intraperitoneal,2000.000000
8623,CC1(C)C=C(C(=O)NCCNC(=O)c2ccccc2C(=O)O)C(C)(C)N1,LD50,357.0,359.43,mg.kg-1,Mus musculus,Intravenous,357.000000
4376,COc1cc([C@@H]2c3cc4c(cc3[C@H](O)[C@H]3COC(=O)[...,LD50,2.0,414.41,ug ml-1,Artemia,None,0.014000
11467,O=C(N/N=C/c1cc(Br)cc(Br)c1O)c1ccc(C(F)(F)F)cc1,LD50,8.0,466.05,ug ml-1,HepG2,In vitro,0.056000
862,O=C(Nc1ccc(C(F)(F)F)cc1)c1ccc2ccccc2c1O,LD50,0.6,331.29,umol/L,THP-1,In vitro,1.391418
5474,CSCC[C@H](NC(=O)[C@@H](NC(=O)CNC(=O)[C@H](CC(C...,LD50,50.6,1519.99,uM,Staphylococcus aureus,None,538.380458


In [13]:
LD_50['LD_50'].notna().sum()

np.int64(11957)

In [15]:
LD_50 = LD_50[pd.notna(LD_50['LD_50'])]
LD_50['Standard Units'] = 'mg.kg-1'

In [33]:
# Сгруппируем все строки по Smiles, типу животного и способу введения вещества и усредним значения LD50 внутри каждой группы
df = LD_50.groupby(by=['Smiles', 'Target Name', 'Administration type']).agg({'LD_50' : 'mean', 'Standard Units': 'first'}).reset_index()
df

,Smiles,Target Name,Administration type,LD_50,Standard Units
0,Br.Br.Cc1cc(-c2ccc(O)cc2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,300.0000,mg.kg-1
1,Br.Br.Cc1cc(-c2cccc(O)c2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0000,mg.kg-1
2,Br.Br.Cc1cc(-c2ccccc2O)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0000,mg.kg-1
3,Br.CC(=O)c1ccc(NC(=O)CCN2CCN(c3ccc(Br)cn3)CC2)cc1,ADMET,Oral,35857.5000,mg.kg-1
4,Br.CC(C)C(N)C(=O)NNc1nncc(-c2ccc(Cl)cc2)n1.O,Mus musculus,Intraperitoneal,178.0000,mg.kg-1
...,...,...,...,...,...
7177,c1cncc(CCc2c[nH]c3ccccc23)c1,NON-PROTEIN TARGET,In vitro,124.4824,mg.kg-1
7178,c1cncc([C@@H]2CCCCN2)c1,ADMET,Intravenous,16.0000,mg.kg-1
7179,c1cncc([C@H]2CCCCN2)c1,ADMET,Intravenous,11.0000,mg.kg-1
7180,c1csc(-c2ccc(-c3ncncc3-c3cccs3)s2)c1,Mus musculus,Oral,160.0000,mg.kg-1


Теперь создадим новый датафрейм, в который будем добавлять данные из всех датафреймоы

In [34]:
chembl_dataset = pd.DataFrame(columns=['Source', 'Smiles', 'Exp. Animal', 'Method of administration', 'LD50', 'LD_50 (a.u.)'])
chembl_dataset['Smiles'] = df['Smiles']
chembl_dataset['Source'] = 'ChEMBL'
chembl_dataset['Exp. Animal'] = df['Target Name']
chembl_dataset['Method of administration'] = df['Administration type']
chembl_dataset['LD50'] = df['LD_50']
chembl_dataset['LD_50 (a.u.)'] = df['Standard Units']

In [35]:
chembl_dataset.shape

(7182, 6)

### Теперь обработаем и добавим к нешему датасету данные по LC50

In [46]:
LC_50 = pd.read_csv('../data/raw/ChEMBL_LC50.csv', delimiter=';')
LC_50.shape

/tmp/ipykernel_49880/1473850495.py:1: DtypeWarning: Columns (30,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  LC_50 = pd.read_csv('../data/raw/ChEMBL_LC50.csv', delimiter=';')


(52793, 47)

In [37]:
# Проверим, есть ли в датасете ПОЛНЫЕ дубли
duplicates = LC_50.duplicated()
duplicated_rows = LC_50[duplicates]
print(duplicates.sum()) # видно, что в датасете есть 224 строки, которые являются дублями
# Удалим их
LC_50.drop_duplicates(inplace=True)

224


In [38]:
# Выберем из датасета только те столбцы, которые понадобятся для дальнейшего анализа
LC_50 = LC_50[['Smiles', 
         'Standard Type',          
         'Standard Value', 
         'Molecular Weight',
         'Standard Units', 
         'Target Name', 
         'Assay Description',
         'Target Organism', 
         'Target Type'
         ]]

In [39]:
print(f'Количество уникальных Smiles: {LC_50['Smiles'].nunique()}') # Из 13626 соединений, для которых приведены Smiles, только 9531 уникальное
# Посмотрим, чем отличаются строки с дублирующимися Smiles
d_smiles = LC_50.duplicated(subset='Smiles', keep=False)
duplicated_rows = LC_50[d_smiles]

Количество уникальных Smiles: 7532


In [40]:
 # определим способ введения вещества в организм из названия публикации об исследовании
LC_50['Administration type'] = LC_50['Assay Description'].apply(adm_type_parsing)
LC_50.drop('Assay Description', axis=1, inplace=True)

In [41]:
# Посмотрим, чем отличаются строки с дублирующимися Smiles
d_smiles = LC_50.duplicated(subset='Smiles', keep=False)
duplicated_rows = LC_50[d_smiles]
duplicated_rows[duplicated_rows['Smiles'] == 'O=c1[nH]cc(F)c(=O)[nH]1']

,Smiles,Standard Type,Standard Value,Molecular Weight,Standard Units,Target Name,Target Organism,Target Type,Administration type
423,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NON-PROTEIN TARGET,NaN,NON-MOLECULAR,In vitro
606,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NON-PROTEIN TARGET,NaN,NON-MOLECULAR,In vitro
1424,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NON-PROTEIN TARGET,NaN,NON-MOLECULAR,In vitro
1425,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NON-PROTEIN TARGET,NaN,NON-MOLECULAR,In vitro
2299,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NON-PROTEIN TARGET,NaN,NON-MOLECULAR,In vitro
...,...,...,...,...,...,...,...,...,...
52764,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,SK-OV-3,Homo sapiens,CELL-LINE,In vitro
52765,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,786-0,Homo sapiens,CELL-LINE,In vitro
52768,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,BT-549,Homo sapiens,CELL-LINE,In vitro
52791,O=c1[nH]cc(F)c(=O)[nH]1,LC50,100000.0,130.08,nM,NCI-H460,Homo sapiens,CELL-LINE,In vitro


In [42]:
# Удалим строки, в которых есть пропуски в ключевых столбцах: Smiles и Standard Value
LC_50 = LC_50[pd.notna(LC_50['Standard Value'])]
# Удалим строки, для которых отсутствуют значения Smiles
LC_50 = LC_50[pd.notna(LC_50['Smiles'])]
# Посмотрим на результат
LC_50.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51585 entries, 0 to 52792
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Smiles               51585 non-null  object 
 1   Standard Type        51585 non-null  object 
 2   Standard Value       51585 non-null  float64
 3   Molecular Weight     51585 non-null  float64
 4   Standard Units       51376 non-null  object 
 5   Target Name          51585 non-null  object 
 6   Target Organism      43728 non-null  object 
 7   Target Type          51585 non-null  object 
 8   Administration type  46382 non-null  object 
dtypes: float64(2), object(7)
memory usage: 3.9+ MB


In [43]:
# Заменим данные в столбце Target Name на данные из столбца Target Organism для тех случаев, где соединение тестировалось на людях 
LC_50[LC_50['Target Organism'] == 'Homo sapiens']['Target Name'] = LC_50['Target Organism']
# Теперь можно удалить столбцы Target Organism и Target Type
LC_50.drop(['Target Organism', 'Target Type'], axis=1, inplace=True)

/tmp/ipykernel_49880/3766261711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LC_50[LC_50['Target Organism'] == 'Homo sapiens']['Target Name'] = LC_50['Target Organism']


In [44]:
# Приведем единицы измерения к одному виду
LC_50['LC_50'] = LC_50.apply(units_calc, df=LC_50, axis=1)
LC_50['LC_50'].notna().sum()

# Отфильтруем все строки, где метрика указана не в mg/kg
LC_50 = LC_50[pd.notna(LC_50['LC_50'])]
LC_50['Standard Units'] = 'mg.kg-1'

df = LC_50.groupby(by=['Smiles', 'Target Name', 'Administration type']).agg({'LC_50' : 'mean', 'Standard Units': 'first'}).reset_index()
df

,Smiles,Target Name,Administration type,LC_50,Standard Units
0,Brc1ccc2[nH]cc(-c3ncnc4ccccc34)c2c1,Unchecked,In vitro,632.701458,mg.kg-1
1,C#C/C=C\CCCCCCCCCCCCC/C=C/C(O)CCCC/C=C\CCCCC#C...,K562,In vitro,140.922600,mg.kg-1
2,C#C/C=C\CCCCCCCCCCCCCC/C=C\CCC/C=C/C(O)CCCCC#C...,K562,In vitro,32.881940,mg.kg-1
3,C#C/C=C\CCCCCCCCCCCCCC/C=C\CCCC/C=C\CCCCC#C[C@...,K562,In vitro,206.980200,mg.kg-1
4,C#C/C=C\CCCCCCCCCCCCCC/C=C\CCCCC(O)/C=C/CCCC#C...,K562,In vitro,98.645820,mg.kg-1
...,...,...,...,...,...
36565,c1cncc(CN2CCN(c3ccc(-c4nc5ccccc5o4)cc3)CC2)c1,ADMET,In vitro,25.932200,mg.kg-1
36566,c1cncc(CN2CCN(c3ccc(-c4nc5ccccc5s4)cc3)CC2)c1,ADMET,In vitro,27.056400,mg.kg-1
36567,c1cncc(CN2CCN(c3ccc(-c4ncco4)cc3)CC2)c1,ADMET,In vitro,22.428000,mg.kg-1
36568,c1csc(CN2CCN(c3ccc(-c4nc5ccccc5o4)cc3)CC2)c1,ADMET,In vitro,26.285000,mg.kg-1


In [45]:
# Создадим временный датафрейм, в который будем добавлять данные, приведенные к стандартному формату
df_temp = pd.DataFrame(columns=['Source', 'Smiles'])
df_temp['Smiles'] = df['Smiles']
df_temp['Source'] = 'ChEMBL'
df_temp['Exp. Animal'] = df['Target Name']
df_temp['Method of administration'] = df['Administration type']
df_temp['LC50'] = df['LC_50']
df_temp['LC_50 (a.u.)'] = df['Standard Units']
df_temp.shape

(36570, 6)

In [27]:
# chembl_dataset = pd.merge(chembl_dataset, df_temp, on=['Source', 'Smiles'], how='outer')

In [27]:
chembl_dataset = pd.concat([chembl_dataset, df_temp], axis=0, ignore_index=True)

In [28]:
chembl_dataset

,Source,Smiles,Exp. Animal,Method of administration,LD50,LD_50 (a.u.),LC50,LC_50 (a.u.)
0,ChEMBL,Br.Br.Cc1cc(-c2ccc(O)cc2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,300.0,mg.kg-1,NaN,NaN
1,ChEMBL,Br.Br.Cc1cc(-c2cccc(O)c2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,mg.kg-1,NaN,NaN
2,ChEMBL,Br.Br.Cc1cc(-c2ccccc2O)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,mg.kg-1,NaN,NaN
3,ChEMBL,Br.CC(=O)c1ccc(NC(=O)CCN2CCN(c3ccc(Br)cn3)CC2)cc1,ADMET,Oral,35857.5,mg.kg-1,NaN,NaN
4,ChEMBL,Br.CC(C)C(N)C(=O)NNc1nncc(-c2ccc(Cl)cc2)n1.O,Mus musculus,Intraperitoneal,178.0,mg.kg-1,NaN,NaN
...,...,...,...,...,...,...,...,...
43747,ChEMBL,c1cncc(CN2CCN(c3ccc(-c4nc5ccccc5o4)cc3)CC2)c1,ADMET,In vitro,NaN,NaN,25.9322,mg.kg-1
43748,ChEMBL,c1cncc(CN2CCN(c3ccc(-c4nc5ccccc5s4)cc3)CC2)c1,ADMET,In vitro,NaN,NaN,27.0564,mg.kg-1
43749,ChEMBL,c1cncc(CN2CCN(c3ccc(-c4ncco4)cc3)CC2)c1,ADMET,In vitro,NaN,NaN,22.4280,mg.kg-1
43750,ChEMBL,c1csc(CN2CCN(c3ccc(-c4nc5ccccc5o4)cc3)CC2)c1,ADMET,In vitro,NaN,NaN,26.2850,mg.kg-1


### Теперь обработаем и добавим к нешему датасету данные по TD50

In [29]:
TD_50 = pd.read_csv('../data/raw/ChEMBL_TD50.csv', delimiter=';')
# Проверим, есть ли в датасете ПОЛНЫЕ дубли
duplicates = TD_50.duplicated()
duplicated_rows = TD_50[duplicates]
print(duplicates.sum()) # видно, что в датасете нет полных дублей
#Удалим их
TD_50.drop_duplicates(inplace=True)

# Выберем из датасета только те столбцы, которые понадобятся для дальнейшего анализа
TD_50 = TD_50[['Smiles', 
         'Standard Type',          
         'Standard Value', 
         'Molecular Weight',
         'Standard Units', 
         'Target Name', 
         'Assay Description',
         'Target Organism', 
         'Target Type'
         ]]

 # определим способ введения вещества в организм из названия публикации об исследовании
TD_50['Administration type'] = TD_50['Assay Description'].apply(adm_type_parsing)
TD_50.drop('Assay Description', axis=1, inplace=True)

# Удалим строки, в которых есть пропуски в ключевых столбцах: Smiles и Standard Value
TD_50 = TD_50[pd.notna(TD_50['Standard Value'])]
# Удалим строки, для которых отсутствуют значения Smiles
TD_50 = TD_50[pd.notna(TD_50['Smiles'])]

# Заменим данные в столбце Target Name на данные из столбца Target Organism для тех случаев, где соединение тестировалось на людях 
TD_50[TD_50['Target Organism'] == 'Homo sapiens']['Target Name'] = TD_50['Target Organism']
# Теперь можно удалить столбцы Target Organism и Target Type
TD_50.drop(['Target Organism', 'Target Type'], axis=1, inplace=True)

# Приведем единицы измерения к одному виду
TD_50['TD_50'] = TD_50.apply(units_calc, df=TD_50, axis=1)
TD_50['TD_50'].notna().sum()

# Отфильтруем все строки, где метрика указана не в mg/kg
TD_50 = TD_50[pd.notna(TD_50['TD_50'])]
TD_50['Standard Units'] = 'mg.kg-1'

df = TD_50.groupby(by=['Smiles', 'Target Name', 'Administration type']).agg({'TD_50' : 'mean', 'Standard Units': 'first'}).reset_index()
df

0


/tmp/ipykernel_48982/1360147180.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TD_50[TD_50['Target Organism'] == 'Homo sapiens']['Target Name'] = TD_50['Target Organism']


,Smiles,Target Name,Administration type,TD_50,Standard Units
0,Brc1ccc(/C=C/c2sc(Nc3ccccc3)n[n+]2-c2ccccc2)cc...,ADMET,In vitro,21.718927,mg.kg-1
1,Brc1cccc(/C=C/c2sc(Nc3ccccc3)n[n+]2-c2ccccc2)c...,ADMET,In vitro,8.140478,mg.kg-1
2,C#CC#CCC/C=C/C=C\C(=O)NCC(C)C,RAW264.7,In vitro,0.350000,mg.kg-1
3,C#CC#CCCCC/C=C\C(=O)NCC(C)C,RAW264.7,In vitro,0.350000,mg.kg-1
4,C(=C/c1sc(Nc2ccccc2)n[n+]1-c1ccccc1)\c1ccccc1....,ADMET,In vitro,8.395141,mg.kg-1
...,...,...,...,...,...
845,Oc1nc(-c2cccnc2)nc2ccccc12,HFF,In vitro,500.057600,mg.kg-1
846,c1ccc(CSc2ncnc3c2cnn3Cc2cccnc2)cc1,HepG2,In vitro,0.280000,mg.kg-1
847,c1ccc(Oc2cccc(CN3CCN(Cc4cccc(Oc5ccccc5)c4)CC3)...,ADMET,In vitro,237.816124,mg.kg-1
848,c1ccc(Oc2cccc(Cn3ccnc3)c2)cc1,ADMET,In vitro,61.323500,mg.kg-1


In [30]:
# Создадим временный датафрейм, в который будем добавлять данные, приведенные к стандартному формату
df_temp = pd.DataFrame(columns=['Source', 'Smiles'])
df_temp['Smiles'] = df['Smiles']
df_temp['Source'] = 'ChEMBL'
df_temp['Exp. Animal'] = df['Target Name']
df_temp['Method of administration'] = df['Administration type']
df_temp['TD50'] = df['TD_50']
df_temp['TD_50 (a.u.)'] = df['Standard Units']

In [31]:
# chembl_dataset = pd.merge(chembl_dataset, df_temp, on=['Source', 'Smiles'], how='outer')

In [31]:
chembl_dataset = pd.concat([chembl_dataset, df_temp], axis=0, ignore_index=True)

In [32]:
chembl_dataset

,Source,Smiles,Exp. Animal,Method of administration,LD50,LD_50 (a.u.),LC50,LC_50 (a.u.),TD50,TD_50 (a.u.)
0,ChEMBL,Br.Br.Cc1cc(-c2ccc(O)cc2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,300.0,mg.kg-1,NaN,NaN,NaN,NaN
1,ChEMBL,Br.Br.Cc1cc(-c2cccc(O)c2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,mg.kg-1,NaN,NaN,NaN,NaN
2,ChEMBL,Br.Br.Cc1cc(-c2ccccc2O)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,mg.kg-1,NaN,NaN,NaN,NaN
3,ChEMBL,Br.CC(=O)c1ccc(NC(=O)CCN2CCN(c3ccc(Br)cn3)CC2)cc1,ADMET,Oral,35857.5,mg.kg-1,NaN,NaN,NaN,NaN
4,ChEMBL,Br.CC(C)C(N)C(=O)NNc1nncc(-c2ccc(Cl)cc2)n1.O,Mus musculus,Intraperitoneal,178.0,mg.kg-1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
44597,ChEMBL,Oc1nc(-c2cccnc2)nc2ccccc12,HFF,In vitro,NaN,NaN,NaN,NaN,500.057600,mg.kg-1
44598,ChEMBL,c1ccc(CSc2ncnc3c2cnn3Cc2cccnc2)cc1,HepG2,In vitro,NaN,NaN,NaN,NaN,0.280000,mg.kg-1
44599,ChEMBL,c1ccc(Oc2cccc(CN3CCN(Cc4cccc(Oc5ccccc5)c4)CC3)...,ADMET,In vitro,NaN,NaN,NaN,NaN,237.816124,mg.kg-1
44600,ChEMBL,c1ccc(Oc2cccc(Cn3ccnc3)c2)cc1,ADMET,In vitro,NaN,NaN,NaN,NaN,61.323500,mg.kg-1


In [35]:
# Проверим, есть ли строки, с общим Smiles, экспериментальным животным и методом введения вещества. 
# Если они есть, то можно "схлопнуть" их в одну строку
check = chembl_dataset.groupby(by=['Smiles', 'Exp. Animal', 'Method of administration']).\
                       agg({'LD50' : 'first', 'LC50': 'first', 'TD50': 'first'}).reset_index()

In [36]:
check

,Smiles,Exp. Animal,Method of administration,LD50,LC50,TD50
0,Br.Br.Cc1cc(-c2ccc(O)cc2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,300.0,NaN,NaN
1,Br.Br.Cc1cc(-c2cccc(O)c2)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,NaN,NaN
2,Br.Br.Cc1cc(-c2ccccc2O)nnc1NCCN1CCOCC1,Mus musculus,Intraperitoneal,200.0,NaN,NaN
3,Br.CC(=O)c1ccc(NC(=O)CCN2CCN(c3ccc(Br)cn3)CC2)cc1,ADMET,Oral,35857.5,NaN,NaN
4,Br.CC(C)C(N)C(=O)NNc1nncc(-c2ccc(Cl)cc2)n1.O,Mus musculus,Intraperitoneal,178.0,NaN,NaN
...,...,...,...,...,...,...
44553,c1cncc([C@H]2CCCCN2)c1,ADMET,Intravenous,11.0,NaN,NaN
44554,c1csc(-c2ccc(-c3ncncc3-c3cccs3)s2)c1,Mus musculus,Oral,160.0,NaN,NaN
44555,c1csc(-c2ccc(-c3ncncc3-c3csc4ccccc34)s2)c1,Mus musculus,Oral,90.0,NaN,NaN
44556,c1csc(CN2CCN(c3ccc(-c4nc5ccccc5o4)cc3)CC2)c1,ADMET,In vitro,NaN,26.2850,NaN


Видно что после группировки осталось 44558 строк, а было 44602 строки, т.е. "схлопнуть" можно только 44 строки. Это совсем немного, так что можно их проигнорировать

In [33]:
# Экспортируем предобработанный датасет в новый csv файл
chembl_dataset.to_csv('../data/processed/ChEMBL_dataset.csv')